In [2]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) # 무작위 데이터 생성
x.shape # (높이&너비 28, 채널 1) * 데이터 10개

(10, 1, 28, 28)

In [27]:
x[0].shape # 첫 번째 데이터

(1, 28, 28)

In [28]:
x[0, 0].shape # 첫 번째 데이터의 첫 채널의 공간 데이터

(28, 28)

In [29]:
# im2col을 사용해 다차원 for문 반복 없이 2차원 행렬 연산
# 한 filter 영역을 앞에서부터 순서대로 1줄로 펴 한 행을 만든다.

from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0) # filter size = 5 * 5
print(col1.shape) # 75 = 필터의 원소 수 (3 * 5 * 5)

(9, 75)


In [31]:
x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


### 합성곱 계층 구현

In [32]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T     # ex. (10, 3, 5, 5) -> (10, 75).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

In [38]:
ex = np.random.rand(4, 5, 6, 7)
ex.shape

(4, 5, 6, 7)

In [39]:
ex.T.shape

(7, 6, 5, 4)

In [41]:
ex.transpose(0, 3, 1, 2).shape

(4, 7, 5, 6)

### 풀링 계층 구현

In [42]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 최댓값
        out = np.max(col, axis=1)
        
        # 성형
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2) # size (0, 1, 2, 3) -> (0, 3, 1, 2)
        
        return out

## CNN 구현